In [4]:
import pandas as pd

In [6]:
full_df = pd.read_csv('dataset/train.csv')
df = full_df.copy()

In [8]:
from sklearn.model_selection import train_test_split

df_train, df_test = train_test_split(df, test_size=0.2)

In [10]:
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.impute import SimpleImputer

def update_train_df(dataframe):
    imputers = {
        'name_imputer': SimpleImputer(strategy='constant', fill_value='Unknown Unknown'),
        'num_imputer': SimpleImputer(strategy='median'),
        'cat_imputer': SimpleImputer(strategy='constant', fill_value='Unknown'),
    }

    num_col = ['Age', 'RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck']
    cat_cols = ['HomePlanet', 'CryoSleep', 'Cabin', 'Destination', 'VIP']
    
    df = dataframe.copy()

    # Impute Part
    df.loc[:, 'Name'] = imputers['name_imputer'].fit_transform(df.loc[:, 'Name'].to_frame())
    df.loc[:, num_col] = imputers['num_imputer'].fit_transform(df.loc[:, num_col])
    df.loc[:, cat_cols] = imputers['cat_imputer'].fit_transform(df.loc[:, cat_cols])

    # Feature engineering Part
    df.loc[:, 'FirstName'] = df.loc[:, 'Name'].apply(lambda x: x.split()[0])
    df.loc[:, 'LastName'] = df.loc[:, 'Name'].apply(lambda x: x.split()[1])

    df.loc[:, 'PassengerGGGG'] = df.loc[:, 'PassengerId'].apply(lambda x: int(x.split('_')[0]))
    df.loc[:, 'PassengerPP'] = df.loc[:, 'PassengerId'].apply(lambda x: int(x.split('_')[1]))
    
    df.loc[:, 'NumberOfFellows'] = df.groupby('PassengerGGGG')['PassengerGGGG'].transform('count')
    df.loc[:, 'IsAlone'] = df.loc[:, 'NumberOfFellows'].apply(lambda x: 1 if x == 1 else 0)

    df['cabin_deck'] = df['Cabin'].apply(lambda x: x.split('/')[0] if x != 'Unknown' else 'U')
    df['cabin_num'] = df['Cabin'].apply(lambda x: int(x.split('/')[1]) if x != 'Unknown' else -1)
    df['cabin_side'] = df['Cabin'].apply(lambda x: x.split('/')[2] if x != 'Unknown' else 'U')

    df.loc[:, 'TotalSpend'] = df.loc[:, ['RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck']].sum(axis=1)
    df['IsSpendingZero'] = (df['TotalSpend'] == 0).astype(int)

    df['n_family_member_in_group'] = (df.groupby(['PassengerGGGG', 'LastName'])['LastName'].transform('count'))
    
    # Removing unnecessary columns
    df.drop(['Cabin', 'Name', 'PassengerId'], inplace=True, axis=1)
    df.drop(['FirstName', 'LastName', 'PassengerGGGG'], inplace=True, axis=1)

    # Encoding 
    cat_data = df.select_dtypes('object')
    cat_data.loc[:, ['CryoSleep', 'VIP']] = cat_data.loc[:, ['CryoSleep', 'VIP']].astype('str')
    
    one_hot_encoder = OneHotEncoder(drop='first', dtype=int)
    
    cat_data_encoded = one_hot_encoder.fit_transform(cat_data)
    encoded_feature_names = one_hot_encoder.get_feature_names_out(cat_data.columns)
    
    cat_encoded = pd.DataFrame(cat_data_encoded.toarray(), columns = encoded_feature_names, index=cat_data.index)
    
    non_cat = df.drop(cat_data.columns, axis=1)
    df = pd.concat([non_cat, cat_encoded], axis=1)
    
    df.loc[:, 'Transported'] = df.loc[:, 'Transported'].astype(int)
    
    return df, imputers, one_hot_encoder

In [12]:
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.impute import SimpleImputer

def update_test_df(dataframe, imputers, one_hot_encoder):
    num_col = ['Age', 'RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck']
    cat_cols = ['HomePlanet', 'CryoSleep', 'Cabin', 'Destination', 'VIP']
    
    df = dataframe.copy()

    # Impute Part
    df.loc[:, 'Name'] = imputers['name_imputer'].transform(df.loc[:, 'Name'].to_frame())
    df.loc[:, num_col] = imputers['num_imputer'].transform(df.loc[:, num_col])
    df.loc[:, cat_cols] = imputers['cat_imputer'].transform(df.loc[:, cat_cols])

    # Feature engineering Part
    df.loc[:, 'FirstName'] = df.loc[:, 'Name'].apply(lambda x: x.split()[0])
    df.loc[:, 'LastName'] = df.loc[:, 'Name'].apply(lambda x: x.split()[1])

    df.loc[:, 'PassengerGGGG'] = df.loc[:, 'PassengerId'].apply(lambda x: int(x.split('_')[0]))
    df.loc[:, 'PassengerPP'] = df.loc[:, 'PassengerId'].apply(lambda x: int(x.split('_')[1]))
    
    df.loc[:, 'NumberOfFellows'] = df.groupby('PassengerGGGG')['PassengerGGGG'].transform('count')
    df.loc[:, 'IsAlone'] = df.loc[:, 'NumberOfFellows'].apply(lambda x: 1 if x == 1 else 0)

    df['cabin_deck'] = df['Cabin'].apply(lambda x: x.split('/')[0] if x != 'Unknown' else 'U')
    df['cabin_num'] = df['Cabin'].apply(lambda x: int(x.split('/')[1]) if x != 'Unknown' else -1)
    df['cabin_side'] = df['Cabin'].apply(lambda x: x.split('/')[2] if x != 'Unknown' else 'U')

    df.loc[:, 'TotalSpend'] = df.loc[:, ['RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck']].sum(axis=1)
    df['IsSpendingZero'] = (df['TotalSpend'] == 0).astype(int)

    df['n_family_member_in_group'] = (df.groupby(['PassengerGGGG', 'LastName'])['LastName'].transform('count'))
    
    # Removing unnecessary columns
    df.drop(['Cabin', 'Name', 'PassengerId'], inplace=True, axis=1)
    df.drop(['FirstName', 'LastName', 'PassengerGGGG'], inplace=True, axis=1)

    # Encoding
    cat_data = df.select_dtypes('object')
    cat_data.loc[:, ['CryoSleep', 'VIP']] = cat_data.loc[:, ['CryoSleep', 'VIP']].astype('str')
        
    cat_data_encoded = one_hot_encoder.transform(cat_data)
    encoded_feature_names = one_hot_encoder.get_feature_names_out(cat_data.columns)
    
    cat_encoded = pd.DataFrame(cat_data_encoded.toarray(), columns = encoded_feature_names, index=cat_data.index)
    
    non_cat = df.drop(cat_data.columns, axis=1)
    df = pd.concat([non_cat, cat_encoded], axis=1)
    
    return df

In [14]:
df_train_updated, train_imputers, train_ohe = update_train_df(df_train) 

df_test_updated = update_test_df(df_test, train_imputers, train_ohe)

X_train = df_train_updated.drop('Transported', axis=1)
y_train = df_train_updated.loc[:, 'Transported']

X_test = df_test_updated.drop('Transported', axis=1)
y_test = df_test_updated.loc[:, 'Transported']

In [ ]:
from lazypredict.Supervised import LazyClassifier

clf = LazyClassifier(verbose=0, ignore_warnings=True, custom_metric=None)
models, predictions = clf.fit(X_train, X_test, y_train, y_test)

models

  0%|          | 0/29 [00:00<?, ?it/s]

In [1]:
!echo $NotebookApp.max_buffer_size

.max_buffer_size
